In [1]:
import tensorflow as tf
from tensorflow import keras
from sklearn.utils import shuffle

import numpy as np

print(tf.__version__)

1.11.0


In [2]:
#MAPPING SENTENCES TO LIBRARY
import json
word_index = keras.datasets.imdb.get_word_index()

#declare trainData as an empty list
trainData = []
trainLabels = []

nonfunctional = 0
functional = 0
    
#temp list for each sentence
tempList = []

with open("Consolidated_data.txt","r") as f:
    
    
    #read data-file line by line
    with open("library.json", "r") as l:
        data = json.load(l)
        
        
        for line in f:
            #find where class begins
            front = line.find("\"class\":\"")
            #find where class ends (where sentence begins)
            end = line.find("\",\"sentence\":\"")
            #substring line based on front and end above
            reqClass = (line[(front+9):end]).lower()

            #sentence
            del tempList[:]
            temp = line.find("\"sentence\":\"")
            sentence = (line[(temp+12):-4]).lower()
            sentence = sentence.translate(None, '.,-\":;~!@#$%^&?[]{}<>`1234567890\\*()').strip()

            if "nonfunctional" in reqClass and (nonfunctional < 1796):
                nonfunctional +=1
                trainLabels.append(0)
                for x in sentence.split(' '):
                    if x in data:
                        tempList.append(data[x])
                    else:
                        tempList.append(1)
                trainData.append(tempList[:])
            elif "functional" in reqClass and (functional < 1796):
                functional +=1
                trainLabels.append(1)
                for x in sentence.split(' '):
                    if x in data:
                        tempList.append(data[x])
                    else:
                        tempList.append(1)
                trainData.append(tempList[:])
            else:
                if (nonfunctional < 1796):
                    nonfunctional +=1
                    trainLabels.append(0)
                    for x in sentence.split(' '):
                        if x in data:
                            tempList.append(data[x])
                        #UNCOMMENTING IS STRONGLY NOT RECOMMENDED
                        #else:
                            #tempList.append(1)
                    trainData.append(tempList[:])
                    

In [3]:
trainData, trainLabels = shuffle(trainData, trainLabels)

In [4]:
from random import randint

testData = []
testLabels = []

nonfunctional = 0
functional = 0

testDataNumber = 100

for x in trainLabels:
    if x == 1 and functional < testDataNumber:
        functional += 1
        testData.append(trainData.pop(x))
        testLabels.append(trainLabels.pop(x))
    else:
        if nonfunctional < testDataNumber:
            nonfunctional += 1
            testData.append(trainData.pop(x))
            testLabels.append(trainLabels.pop(x))

In [5]:
print(trainLabels.count(1))
print(trainLabels.count(0))
print(testLabels.count(1))
print(testLabels.count(0))

1695
1697
101
99


In [6]:
#randomizing data in two arrays without disrupting the order
testData, testLabels = shuffle(testData, testLabels)

In [7]:
#sanity check
#those should NOT be the same
#print testData[0]
print trainData[0]

[255, 252, 62, 6, 676, 104]


In [8]:
# A dictionary mapping words to an integer index
word_index = json.load(open('library.json', 'r'))
#word_index = keras.datasets.imdb.get_word_index()

# The first indices are reserved
word_index = {k:(v) for k,v in word_index.items()} 
word_index["<PAD>"] = 0
word_index["<UNK>"] = 1 # unknown
word_index["<START>"] = 2

reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

In [9]:
decode_review(trainData[0])

u'emergency contact name and phone number'

In [10]:
trainData = keras.preprocessing.sequence.pad_sequences(trainData,
                                                        value=word_index["<PAD>"],
                                                        padding='post',
                                                        maxlen=32)

testData = keras.preprocessing.sequence.pad_sequences(testData,
                                                       value=word_index["<PAD>"],
                                                       padding='post',
                                                       maxlen=32)

In [11]:
vocab_size = 5000

model = keras.Sequential()
model.add(keras.layers.Embedding(vocab_size, 16))
model.add(keras.layers.GlobalAveragePooling1D())
model.add(keras.layers.Dense(16, activation=tf.nn.relu))
model.add(keras.layers.Dense(1, activation=tf.nn.sigmoid))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 16)          80000     
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 16)                272       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 80,289
Trainable params: 80,289
Non-trainable params: 0
_________________________________________________________________


In [12]:
model.compile(optimizer=tf.train.AdamOptimizer(),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [13]:
x_val = trainData[2500:]
partial_x_train = trainData[:2500]

y_val = trainLabels[2500:]
partial_y_train = trainLabels[:2500]

In [14]:
print(model.evaluate(testData, testLabels))

200/200 [==============================] - 0s 988us/step
[0.6937040209770202, 0.48]


In [15]:
history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=50,
                    batch_size=512,
                    validation_data=(x_val, y_val),
                    verbose=1)

Train on 2500 samples, validate on 892 samples
Epoch 1/50
2500/2500 [==============================] - 1s 236us/step - loss: 0.6933 - acc: 0.4992 - val_loss: 0.6933 - val_acc: 0.4765
Epoch 2/50
2500/2500 [==============================] - 0s 26us/step - loss: 0.6926 - acc: 0.5676 - val_loss: 0.6928 - val_acc: 0.5482
Epoch 3/50
2500/2500 [==============================] - 0s 24us/step - loss: 0.6920 - acc: 0.6244 - val_loss: 0.6923 - val_acc: 0.5964
Epoch 4/50
2500/2500 [==============================] - 0s 23us/step - loss: 0.6910 - acc: 0.6888 - val_loss: 0.6915 - val_acc: 0.6256
Epoch 5/50
2500/2500 [==============================] - 0s 27us/step - loss: 0.6896 - acc: 0.6988 - val_loss: 0.6904 - val_acc: 0.6379
Epoch 6/50
2500/2500 [==============================] - 0s 27us/step - loss: 0.6880 - acc: 0.6940 - val_loss: 0.6890 - val_acc: 0.6278
Epoch 7/50
2500/2500 [==============================] - 0s 26us/step - loss: 0.6860 - acc: 0.6768 - val_loss: 0.6874 - val_acc: 0.6211
Epoch 8

In [16]:
results = model.evaluate(testData, testLabels)
print(results)

200/200 [==============================] - 0s 74us/step
[0.6001617813110351, 0.66]
